## Keypoint Matching

### Import packages

In [1]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from scipy.optimize import linear_sum_assignment

### Check GPU availability, using nvidia-smi 

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

### Load image and query description

In [3]:
image_des = torch.load("image_description.pth")
query_des = torch.load("query_description.pth")
print(image_des.shape)
print(query_des.shape)

torch.Size([136, 20, 128])
torch.Size([34, 20, 128])


### many-to-mant and one-to-one matching

In [12]:
def one2one_cost(query_des, image_des):
    cost = torch.zeros(20, 20)
    simi = torch.zeros(20, 20)
    for i in range(20):
        for j in range(20):
            cost[i, j] = torch.dist(query_des[i], image_des[j], 2)
            simi[i, j] = torch.exp(-1 * cost[i, j])
    row_ind, col_ind = linear_sum_assignment(cost.cpu().numpy())
    
    return simi[row_ind, col_ind].sum()

def many2many_cost(query_des, image_des, threshold=0.06):
    simi = torch.zeros(20, 20)
    for i in range(20):
        for j in range(20):
            simi[i, j] = torch.exp(-1*torch.dist(query_des[i], image_des[j], 2))
    simiNorm = torch.sqrt(torch.sum(simi * simi))
    simi_w = simi / simiNorm
    threshold = max(threshold, (simi_w.max() - simi_w.min())/2)
    simi_w = simi_w.gt(threshold).float()
    
    return (simi*simi_w).sum()

### Get total similarity of matching

In [13]:
def similarity(cost, query_des, image_des):
    simi = torch.zeros(34, 136)
    for i in range(34):
        for j in range(136):
            simi[i, j] = cost(query_des[i], image_des[j])
    return simi

In [14]:
simi_one_one = similarity(one2one_cost, query_des, image_des)

In [15]:
simi_many_many = similarity(many2many_cost, query_des, image_des)

In [16]:
torch.save(simi_one_one, "similarity_one.pth")
torch.save(simi_many_many, "similarity_many.pth")

In [17]:
simi_one_one.shape

torch.Size([34, 136])

In [18]:
simi_many_many.shape

torch.Size([34, 136])